In [31]:
import requests
import re
import json
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup

In [34]:
film_id = 'f_0B8sq'
letterboxd_url = 'https://boxd.it/3ETA'

In [35]:
r = requests.get(letterboxd_url)
r


<Response [200]>

In [36]:
redirected_url = r.url
if letterboxd_url != redirected_url:
    r = requests.get(redirected_url)

In [37]:
soup = BeautifulSoup(r.content, 'lxml')

In [38]:
og_url = soup.find('meta', {'property': 'og:url'}).get('content')
film = og_url.split('/')[-2]
try:
    year = int(list(re.search(r'\((.*?)\)', soup.find('meta', {'property': 'og:title'}).get('content')).groups())[0])
except:
    year = int(datetime.now().strftime('%Y')) + 2
genre_list = [x.get('href').replace('/films/genre/', '').replace('/', '') for x in soup.findAll('a', {'class':'text-slug'}) if 'genre' in str(x.get('href'))]

rating_dict = json.loads(soup.find('script', {'type':"application/ld+json"}).string.split('\n')[2]).get('aggregateRating')
try:
    rating_mean = rating_dict.get('ratingValue')
except:
    rating_mean = np.nan
try:
    rating_count = rating_dict.get('ratingCount')
except:
    rating_count = np.nan

In [45]:
rating_count

1074

In [46]:
r = requests.get('https://letterboxd.com/film/{}/members/rated/.5-5/'.format(film))

In [47]:
soup = BeautifulSoup(r.content, 'lxml')

In [48]:
metrics_dict = {}
for i in ['members', 'fans', 'likes', 'reviews', 'lists']:
    href_str = '/film/{}/{}/'.format(film, i)
    try:
        metric_string = soup.find('a', {'class': 'tooltip', 'href':href_str}).get('title')
        metric = int(metric_string[:metric_string.find('\xa0')].replace(',', ''))
    except:
        metric = 0
    metrics_dict[i] = metric

In [49]:
metrics_dict

{'members': 2348, 'fans': 0, 'likes': 447, 'reviews': 316, 'lists': 501}

In [50]:
r = requests.get('https://letterboxd.com/esi/film/{}/stats/'.format(film))
soup = BeautifulSoup(r.content, 'lxml')

In [52]:
try:
    top_ = int(soup.find('a', {'class': 'has-icon icon-top250 icon-16 tooltip'}).text)
except:
    top_ = None
letterboxd_info_dict = {
    'FILM_ID': film_id,
    'FILM_WATCH_COUNT': metrics_dict['members'],
    'FILM_FAN_COUNT': metrics_dict['fans'],
    'FILM_LIKES_COUNT': metrics_dict['likes'],
    'FILM_REVIEW_COUNT': metrics_dict['reviews'],
    'FILM_LIST_COUNT': metrics_dict['lists'],
    'FILM_TOP_250': top_,
    'FILM_RATING': rating_mean,
    'FILM_RATING_COUNT': rating_count,
}

In [54]:
film_genre_dict = {
        'FILM_ID': film_id,
        'FILM_GENRE':genre_list[0],
        'ALL_FILM_GENRES': '/'.join(genre_list)
    }


IndexError: list index out of range